In [1]:
import requests
import pandas as pd
from datetime import datetime
import time
from typing import List, Dict
import logging

class PBPStatsAPI:
    def __init__(self):
        self.base_url = "https://api.pbpstats.com/get-game-logs/nba?"
        self.season_types = ["Regular Season", "Playoffs"]
        
    def get_season_years(self, start_year: int = 2013) -> List[str]:
        """Generate season strings from start_year to current year."""
        current_year = 2024
        return [f"{year}-{str(year + 1)[2:]}" for year in range(start_year, current_year+1)]
    
    def get_team_game_logs(self, team_id: str, start_year: int = 2013) -> pd.DataFrame:
        """
        Fetch all game logs for a specific team since start_year.
        
        Args:
            team_id (str): The team ID in PBP Stats format
            start_year (int): Starting year (default: 2001)
            
        Returns:
            pd.DataFrame: DataFrame containing all game logs
        """
        all_games = []
        seasons = self.get_season_years(start_year)
        
        for season in seasons:
            for season_type in self.season_types:
                if season_type!='Playoffs':
                    try:
                        params = {
                            "Season": season,
                            "SeasonType": season_type,
                            "EntityId": team_id,
                            "EntityType": "Team"
                        }
                        
                        response = requests.get(self.base_url, params=params)
             
                        
                        data_response = response.json()
                            
                        raw_data= data_response["multi_row_table_data"]
                        games_data=pd.DataFrame(raw_data)
         
                        games_data['Season'] = season
                        games_data['year']=int(season.split('-')[0])+1
                        games_data['SeasonType'] = season_type
                        all_games.append(games_data)
                                
                        # Respect API rate limits
                        time.sleep(2)
                        
                    except requests.exceptions.RequestException as e:
                        logging.error(f"Error fetching data for {team_id} in {season} {season_type}: {str(e)}")
                        continue
                    
        return pd.concat(all_games) if all_games else pd.DataFrame()

def fetch_all_teams_game_logs(team_ids: List[str], start_year: int = 2013) -> Dict[str, pd.DataFrame]:
    """
    Fetch game logs for multiple teams.
    
    Args:
        team_ids (List[str]): List of team IDs
        start_year (int): Starting year for data collection
        
    Returns:
        Dict[str, pd.DataFrame]: Dictionary with team IDs as keys and their game logs as values
    """
    api = PBPStatsAPI()
    team_games = {}
    
    for team_id in team_ids:
        logging.info(f"Fetching game logs for team {team_id}")
        team_games[team_id] = api.get_team_game_logs(team_id, start_year)
        
    return team_games

# Example usage
if __name__ == "__main__":
    # Example team IDs (you'll need to use the correct IDs from PBP Stats)
    team_ids = ["LAL", "BOS", "GSW"]  # Replace with actual team IDs
    df =pd.read_csv('index_master.csv')
    df=df[df.year>2013]
    df=df[df.team!='TOT']
    team_ids=df['team_id'].unique().tolist()

    # Configure logging
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )
    
    # Fetch game logs
    team_game_logs = fetch_all_teams_game_logs(team_ids,start_year=2024)
    
    # Example: Save to CSV files
    for team_id, games_df in team_game_logs.items():
        
        if not games_df.empty:
            for year in range(2024,2026):
                team_df=games_df[games_df.year==year]
                team_df.to_csv(f"team/{year}/{team_id}.csv", index=False)
            logging.info(f"Saved game logs for {team_id}")

2024-11-24 14:45:42,813 - INFO - Fetching game logs for team 1610612761
2024-11-24 14:45:45,514 - INFO - Fetching game logs for team 1610612758
2024-11-24 14:45:48,250 - INFO - Fetching game logs for team 1610612760
2024-11-24 14:45:51,023 - INFO - Fetching game logs for team 1610612766
2024-11-24 14:45:53,788 - INFO - Fetching game logs for team 1610612749
2024-11-24 14:45:56,648 - INFO - Fetching game logs for team 1610612753
2024-11-24 14:45:59,524 - INFO - Fetching game logs for team 1610612740
2024-11-24 14:46:02,401 - INFO - Fetching game logs for team 1610612752
2024-11-24 14:46:05,206 - INFO - Fetching game logs for team 1610612757
2024-11-24 14:46:07,982 - INFO - Fetching game logs for team 1610612755
2024-11-24 14:46:10,667 - INFO - Fetching game logs for team 1610612754
2024-11-24 14:46:13,368 - INFO - Fetching game logs for team 1610612748
2024-11-24 14:46:16,018 - INFO - Fetching game logs for team 1610612763
2024-11-24 14:46:18,778 - INFO - Fetching game logs for team 161

In [2]:
team_game_logs

{'1610612761':    Minutes  OffPoss  DefPoss  PenaltyOffPoss  PenaltyDefPoss  \
 0    48:00      100      101              42              38   
 1    48:00      101      102              56              43   
 2    48:00       99      100               4              19   
 3    53:00      111      111              25              50   
 4    48:00      101      102              36              30   
 5    48:00      102      101              17              35   
 6    53:00      113      115              18              39   
 7    48:00      103      101              22              46   
 8    48:00       93       93               9              39   
 9    48:00       96       95              25              21   
 10   48:00       90       92              16              26   
 11   48:00       97       99              18               9   
 12   48:00      100      100              35               7   
 13   53:00       97       99              16              25   
 14   48:00